## Postprocess and Analyze

In [ ]:
import logging
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.INFO)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.morphology import medial_axis

import src.measure.isolines as isolines_calculator
from src.measure.curvature import calc_local_curvature, calc_local_curvature_old
import src.simulation.plotting as plotting
import src.simulation.analysis as analysis
import src.simulation.postprocessing as postprocessing

In [ ]:
test = False

do_postprocessing = False
if do_postprocessing:
    df = pd.read_csv("results/simulation_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    simulation_results = df.to_dict(orient="records")
    postprocessing_results = postprocessing.postprocess(simulation_results)
    pd.DataFrame(postprocessing_results).to_csv("results/postprocessing_results.csv")

do_analysis = True
analysis_params = {
    "iso_value":0.98,
    "curvature_smoothing_stride": 2,
}
if do_analysis:
    df = pd.read_csv("results/postprocessing_results.csv", index_col=0)
    if test:
        df = df.tail(5)
    postprocessing_results = df.to_dict(orient="records")
    analysis_results = analysis.analyze(postprocessing_results, **analysis_params)
    df = pd.DataFrame(analysis_results)
    df.to_csv("results/analysis_results.csv")

## Plot

In [ ]:
df = pd.read_csv("results/analysis_results.csv", index_col=0)
# remove runs with skeleton_length smaller than 500
df = df[df["skeleton_length"] > 500]
# df = df[(df["std_lambda"] == 5.05e8/4) | (df["homogeneous"] == True)]
# is_in_wanted_lengthscale = df["lengthscale_lambda"]== 1
# is_in_wanted_lengthscale = np.logical_or(is_in_wanted_lengthscale, df["lengthscale_lambda"]== 5)
# is_in_wanted_lengthscale = np.logical_or(is_in_wanted_lengthscale, df["lengthscale_lambda"]== 1000)

plots = ["volume", "count", "fractal_dimension",
         "isoline_length", "skeleton_length",
         "total_curvature", "skeleton_y_max",
         "sign_changes", "max_curvature", "max_curvature_smoothed", "sign_changes_smoothed",
         "isoline_n_points", "isoline_mean_segment_length"]

df_sub = df#[is_in_wanted_lengthscale]
plotting.plot(df_sub, create_subplots=True, plots=plots)

In [ ]:
# Special plot for the curvatures
n_std_lambdas = len(df["std_lambda"].unique())
n_lengthscale_lambdas = len(df["lengthscale_lambda"].unique())
n_plots = n_std_lambdas * n_lengthscale_lambdas
# give each lengthscale_lambda a different color
colors = plt.cm.rainbow(np.linspace(0, 1, n_lengthscale_lambdas))
# plt.subplots(n_plots, 1, figsize=(10, 20), tight_layout=True)
plt.figure()

for i, std_lambda in enumerate(df["std_lambda"].unique()):
    df_std_lambda = df[df["std_lambda"] == std_lambda]
    for j, lengthscale_lambda in enumerate(df_std_lambda["lengthscale_lambda"].unique()):
        df_lengthscale_lambda = df_std_lambda[df_std_lambda["lengthscale_lambda"] == lengthscale_lambda]
        cs = []
        for index, row in df_lengthscale_lambda.iterrows():
            c = np.load(row["curvatures"])
            cs.append(c)
        # scale c to [0, 1]
        c = np.mean(cs, axis=0)
        # c = (c - np.min(c)) / (np.max(c) - np.min(c))
        # plt.subplot(n_plots, 1, i * n_lengthscale_lambdas + j + 1)
        if len(cs) > 1:
            plt.semilogy(c, label=f"lengthscale_lambda={lengthscale_lambda:.2E}", color=colors[j])
        # plt.legend()

# make a legend from the color patches
patches = [plt.plot([], [], marker="o", ms=10, ls="", mec=None, color=colors[i], label=f"lengthscale_lambda={lengthscale_lambda:.2E}")[0] for i, lengthscale_lambda in enumerate(df["lengthscale_lambda"].unique())]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

# plt.legend()
plt.xlabel("Smoothing cycles")
plt.ylabel("Curvature")
plt.show()


## Look at the cracks, the postprocessed data and the visual analysis results:

In [ ]:
from skimage.morphology import skeletonize
for index, row in df.iterrows():
    OP_bw = row["OP_01_bw"]
    OP_bw = np.load(OP_bw)
    OP_bw = OP_bw > 0.5
    OP_01 = np.load(row["OP_01"])
    skel, distance = medial_axis(OP_bw, return_distance=True)
    skeleton = skeletonize(OP_bw)
    dist_on_skel = distance * skeleton

    summed_width = np.sum(dist_on_skel)
    print(f"summed_width: {summed_width}")
    print(f"width_variance: {np.var(dist_on_skel)}")

    plt.imshow(dist_on_skel, cmap='magma')
    plt.colorbar()

    plt.contour(OP_01, [0.98], colors='w')
    plt.show()

In [ ]:
for index, row in df.iterrows():
    # Draw the original image
    # OP_01 = row["OP_01_bw"]
    # arr = np.load(OP_01)
    # plt.imshow(arr, cmap=plt.cm.gray)
    # plt.show()

    # Draw the skeleton
    skeleton = np.load(row["skeleton"])
    plt.imshow(skeleton, cmap=plt.cm.gray)
    plt.show()

    print('Skeleton length:', row["skeleton_length"])


In [ ]:
# TODO: Plot the fractures and check the results for different iso_values

## Plot Isolines

In [ ]:
for index, row in df.iterrows():
    OP_bw = np.load(row["OP_01_bw"])
    plt.imshow(OP_bw, cmap=plt.cm.gray)
    OP_01_isolines = isolines_calculator.isoline(OP_bw, iso_value=row["iso_value"])
    for isoline in OP_01_isolines:
        plt.plot(isoline[:, 1], isoline[:, 0], linewidth=2, color='red')
    plt.show()
    
    print("The total curvature is:", row["curvature"])
    for smoothing_cyle in range(1, row["curvature_smoothing_cycles"]+1):
        curvature_smoothed = row[f"curvature_smoothed_{smoothing_cyle}"]
        print(f"The curvature after {smoothing_cyle} smoothing cycles is:", curvature_smoothed)

for index, row in df.iterrows():
    # select where std_lambda == 5.05e8
    sub_df = df[df["std_lambda"]==5.05e8]
    strides = [1,3,5,10,25]
    for stride in strides:
        plt.plot(sub_df["lengthscale_lambda"])

In [ ]:
# random_indices = np.random.randint(0, len(df), 10)
all_indices = np.arange(len(df))
random_indices = all_indices

y_min, y_max = -150, 150 #df["structured_mesh_min_y"].min(), df["structured_mesh_max_y"].max()

for i in random_indices:
    # Select row by using the column "run" which contains the run number which has the same range as the index
    df_row = df.iloc[i]
    OP_01 = np.load(df_row["OP_01"])
    isolines = isolines_calculator.isoline(OP_01, iso_value=df_row["iso_value"])
    # plt.figure()
    # curvatures = []
    # for isoline in isolines:
    #     isoline_curvature = curvature.calc_curvature(isoline, stride=20)
    #     curvatures.append(isoline_curvature)

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax2.set_ylabel('Curvature', color='b')
    ax1.set_ylabel('y')
    ax1.set_ylim(y_min, y_max)
    stride = 3
    # ax1.set_ylim(y_min, y_max)
    curvature_sum_sum = 0
    for isoline in isolines[0:1]:
        # if np.max(isoline[:, 0]) - np.min(isoline[:, 0]) < 100:
        #     continue
        isoline_curvature_local = calc_local_curvature(isoline, stride=stride)
        curvature_sum = np.sum(np.abs(isoline_curvature_local))
        curvature_sum_sum += curvature_sum
        # x = np.linspace(0, 1000, len(isoline_curvature_local))
        ax2.plot(isoline[::stride,1], isoline_curvature_local, color='b')
        ax1.plot(isoline[::stride,1], isoline[::stride,0], color='r')
    ax1.set_title(f"curvature_sum={curvature_sum_sum}") #, curvature={isoline_curvature:.2f}")
    plt.show()

## Plot Volume